<a href="https://colab.research.google.com/github/babalukadam/data-engineering-pract_project2601/blob/main/extract_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PATH"] += f":{os.environ['SPARK_HOME']}/bin"


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,551 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,851 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.3 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:14 http://security.u

In [ ]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("TXT_to_CSV") \
    .getOrCreate()

# Path to your TXT file
input_path = "/customer_flatfile_100.txt"
# output_path = "C:/Users/HP/Desktop/abc/New folder/output_csv.csv"

# Read TXT file as DataFrame (pipe-delimited)
df = spark.read.option("header", True) \
    .option("delimiter", "|") \
    .csv(input_path)

# Show sample
df.show(20)

FileNotFoundError: [Errno 2] No such file or directory: '/content/spark-3.5.0-bin-hadoop3/./bin/spark-submit'

# New Section

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StructType, StructField, StringType

spark = SparkSession.builder \
    .appName("Parse_Null_Separated_File") \
    .getOrCreate()

input_path = "/input_file.txt"

raw_df = spark.read.text(input_path)

# ---------------------------------------------------------
# FILTER ONLY VALID PRODUCT RECORDS
# A real record always starts with: \x00 + number + \x00
# Example: \x001\x00 or \x002\x00 etc
# ---------------------------------------------------------
filtered_df = raw_df.filter(col("value").startswith("\x00"))

# ---------------------------------------------------------
# UDF to parse real records
# ---------------------------------------------------------
def parse_record(line):
    if not line or not line.startswith("\x00"):
        return [""] * 12

    parts = line.split("\x00")
    parts = [p for p in parts if p.strip() != ""]

    serial_no = parts[0] if len(parts) > 0 else ""
    part_no = parts[1] if len(parts) > 1 else ""
    name_eng = parts[2] if len(parts) > 2 else ""
    lang1 = parts[3] if len(parts) > 3 else ""
    lang2 = parts[4] if len(parts) > 4 else ""
    lang3 = parts[5] if len(parts) > 5 else ""
    lang4 = parts[6] if len(parts) > 6 else ""
    lang5 = parts[7] if len(parts) > 7 else ""

    remaining = parts[8:]

    part_no_other = remaining[0] if len(remaining) > 0 else ""
    reference_no = remaining[1] if len(remaining) > 1 else ""
    add_info = remaining[2] if len(remaining) > 2 else ""

    extra = ""
    if len(remaining) > 3:
        extra = "_".join(remaining[3:])

    return [
        serial_no,
        part_no,
        name_eng,
        lang1,
        lang2,
        lang3,
        lang4,
        lang5,
        part_no_other,
        reference_no,
        add_info,
        extra
    ]

schema = StructType([
    StructField("Serial Number", StringType(), True),
    StructField("Part Number", StringType(), True),
    StructField("Part Name English", StringType(), True),
    StructField("Part Name Language 1", StringType(), True),
    StructField("Part Name Language 2", StringType(), True),
    StructField("Part Name Language 3", StringType(), True),
    StructField("Part Name Language 4", StringType(), True),
    StructField("Part Name Language 5", StringType(), True),
    StructField("Part Number in Other Format", StringType(), True),
    StructField("Reference Number", StringType(), True),
    StructField("Additional Information", StringType(), True),
    StructField("Extra Data", StringType(), True)
])

parse_udf = udf(parse_record, schema)

parsed_df = filtered_df.withColumn("parsed", parse_udf(col("value"))) \
                       .select("parsed.*")
parsed_df.show()



+-------------+-----------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+----------+
|Serial Number|Part Number|Part Name English|Part Name Language 1|Part Name Language 2|Part Name Language 3|Part Name Language 4|Part Name Language 5|Part Number in Other Format|Reference Number|Additional Information|Extra Data|
+-------------+-----------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+----------+
+-------------+-----------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+----------+



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

# 1. Read raw file as a single column
df_raw = spark.read.text("/input_file.txt")

# 2. Remove binary garbage (anything not printable)
clean_df = df_raw.withColumn(
    "clean",
    F.regexp_replace("value", r"[^\\x00-\\x7F]", "")   # keep only ASCII
).withColumn(
    "clean",
    F.regexp_replace("clean", r"[\\xff\\x00]{2,}", "\\x00")  # collapse repeated nulls
)

# 3. Split by NULL separator
split_df = clean_df.withColumn("parts", F.split("clean", "\\\\x00"))

# 4. UDF to map fields into your schema
def parse_record(parts):
    parts = [p for p in parts if p.strip() != ""]
    if len(parts) < 10:
        return None

    serial = parts[0]
    part_no = parts[1]
    english = parts[2]
    lang1 = parts[3]
    lang2 = parts[4]
    lang3 = parts[5]
    lang4 = parts[6]
    lang5 = parts[7]
    other_format = parts[8]
    reference = parts[9]
    additional = parts[10] if len(parts) > 10 else None

    # Remaining values → Extra Data
    extra = "_".join(parts[11:]) if len(parts) > 11 else None

    return (serial, part_no, english, lang1, lang2, lang3, lang4, lang5,
            other_format, reference, additional, extra)

schema = T.StructType([
    T.StructField("Serial Number", T.StringType()),
    T.StructField("Part Number", T.StringType()),
    T.StructField("Part Name English", T.StringType()),
    T.StructField("Part Name Language 1", T.StringType()),
    T.StructField("Part Name Language 2", T.StringType()),
    T.StructField("Part Name Language 3", T.StringType()),
    T.StructField("Part Name Language 4", T.StringType()),
    T.StructField("Part Name Language 5", T.StringType()),
    T.StructField("Part Number in Other Format", T.StringType()),
    T.StructField("Reference Number", T.StringType()),
    T.StructField("Additional Information", T.StringType()),
    T.StructField("Extra Data", T.StringType())
])

parse_udf = F.udf(parse_record, schema)

# 5. Apply transformation
final_df = split_df.withColumn("parsed", parse_udf("parts")).select("parsed.*")

# 6. Remove null rows (binary garbage)
final_df = final_df.filter(F.col("Serial Number").isNotNull())

# final_df.show(truncate=False)


In [ ]:
final_df.count()

0

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

# 1. Read file as a single string
raw = spark.read.text("/input_file.txt").selectExpr("concat_ws('', collect_list(value)) as data").first()[0]

# 2. Keep only the valid text region (stop before binary garbage)
# Valid text always contains printable characters + \x00
import re
valid_text = re.findall(r'[\x00-\x7F]+', raw)[0]   # take first ASCII block

# 3. Split by NULL separator
parts = [p for p in valid_text.split("\x00") if p.strip() != ""]

# 4. Convert flat list into records
records = []
i = 0

while i < len(parts):

    # ✅ SAFETY CHECK: ensure enough fields exist
    if i + 9 >= len(parts):
        break

    serial = parts[i]
    part_no = parts[i+1]
    english = parts[i+2]
    lang1 = parts[i+3]
    lang2 = parts[i+4]
    lang3 = parts[i+5]
    lang4 = parts[i+6]
    lang5 = parts[i+7]
    other_format = parts[i+8]
    reference = parts[i+9]

    # Additional info may or may not exist
    additional = parts[i+10] if (i+10) < len(parts) else None

    # ✅ Collect extra data until next serial number (digit-only)
    extra = []
    j = i + 11
    while j < len(parts) and not parts[j].isdigit():
        extra.append(parts[j])
        j += 1

    extra_data = "_".join(extra) if extra else None

    records.append((
        serial, part_no, english, lang1, lang2, lang3, lang4, lang5,
        other_format, reference, additional, extra_data
    ))

    # Move to next record
    i = j

# 5. Create Spark DataFrame
schema = T.StructType([
    T.StructField("Serial Number", T.StringType()),
    T.StructField("Part Number", T.StringType()),
    T.StructField("Part Name English", T.StringType()),
    T.StructField("Part Name Language 1", T.StringType()),
    T.StructField("Part Name Language 2", T.StringType()),
    T.StructField("Part Name Language 3", T.StringType()),
    T.StructField("Part Name Language 4", T.StringType()),
    T.StructField("Part Name Language 5", T.StringType()),
    T.StructField("Part Number in Other Format", T.StringType()),
    T.StructField("Reference Number", T.StringType()),
    T.StructField("Additional Information", T.StringType()),
    T.StructField("Extra Data", T.StringType())
])

final_df = spark.createDataFrame(records, schema)
final_df.show(truncate=False)

In [ ]:
binary_df = spark.read.format("binaryFile").load("/input_file.txt")
raw_bytes = binary_df.select("content").first()[0]

In [ ]:
raw_text = raw_bytes.decode("latin1")

In [ ]:
raw_text = raw_text.replace("\\x00", "\x00")

In [ ]:

import re
valid_text = re.findall(r'[\x00-\x7F]+', raw_text)[0]


In [ ]:
parts = [p for p in valid_text.split("\x00") if p.strip() != ""]

In [ ]:
print(parts)

['1', 'X530108146000', 'INTERNAL RETAINING RING', "ANNEAU D\\'ARRET POUR ALESAGE", 'INNENHALTERING', 'ANELLO DI SICUREZZA INTERNO', 'ANILLO DE RETENCION INTERNO', 'PIER\\xc5\\x9aCIE\\xc5\\x83 USTALAJ\\xc4\\x84CY WEWN\\xc4\\x98TRZNY', 'X530.108.146.000', '72311106', 'DIN472 - 90X3', '2', '139100150130', 'SHIM RING', 'ANNEAU ENTRETOISE', 'UNTERLEGRING', 'DISCO SPESSORE', 'ANILLO DE CALCE', 'PIER\\xc5\\x9aCIE\\xc5\\x83 REGULACYJNY', '139.100.150.130', '72316227', '0,10 MM', '139100150131', '139.100.150.131', '72316228', '0,15 MM', '139100150132', '139.100.150.132', '72316229', '0,25 MM', '139100150133', '139.100.150.133', '72316230', '0,30 MM', '139100150134', '139.100.150.134', '72316231', '0,80 MM', '3', 'X548967266000', 'O-RING', 'JOINT TORIQUE', 'GIUNTO DI TENUTA', 'JUNTA TOROIDAL', 'X548.967.266.000', '72313499', '82,14X3,53-NBR 70 FWN56402', '4', '178100010020', 'SEALING COVER', 'CACHE ETANCHE', 'DICHTDECKEL', 'COPERTURA TENUTA', 'CUBIERTA SELLANTE', 'POKRYWA USZCZELNIAJ\\xc4\\x84CA

In [ ]:
# records = []
# current = []

# for p in parts:
#     if p.isdigit() and current:
#         records.append(current)
#         current = [p]
#     else:
#         current.append(p)

# if current:
#     records.append(current)

In [ ]:
records = []
current = []

def is_new_record(idx):
    # Must have next field
    if idx + 1 >= len(parts):
        return False

    this_field = parts[idx]
    next_field = parts[idx + 1]

    # Serial number must be digits
    if not this_field.isdigit():
        return False

    # Next field must be alphanumeric (part number)
    if next_field.isdigit():
        return False  # numeric → not a part number

    # Next field must not be measurement
    if "MM" in next_field or "," in next_field:
        return False

    return True

i = 0
while i < len(parts):
    if is_new_record(i):
        if current:
            records.append(current)
        current = [parts[i]]
    else:
        current.append(parts[i])
    i += 1

if current:
    records.append(current)

In [ ]:
print(len(records))
print(records[0])

86
['1', 'X530108146000', 'INTERNAL RETAINING RING', "ANNEAU D\\'ARRET POUR ALESAGE", 'INNENHALTERING', 'ANELLO DI SICUREZZA INTERNO', 'ANILLO DE RETENCION INTERNO', 'PIER\\xc5\\x9aCIE\\xc5\\x83 USTALAJ\\xc4\\x84CY WEWN\\xc4\\x98TRZNY', 'X530.108.146.000']


In [ ]:
final = []
for r in records:
    serial = r[0]
    part_no = r[1] if len(r)>1 else None
    english = r[2] if len(r)>2 else None
    lang1 = r[3] if len(r)>3 else None
    lang2 = r[4] if len(r)>4 else None
    lang3 = r[5] if len(r)>5 else None
    lang4 = r[6] if len(r)>6 else None
    lang5 = r[7] if len(r)>7 else None
    other_format = r[8] if len(r)>8 else None
    reference = r[9] if len(r)>9 else None
    additional = r[10] if len(r)>10 else None
    extra = "_".join(r[11:]) if len(r)>11 else None

    final.append((serial, part_no, english, lang1, lang2, lang3, lang4, lang5,
                  other_format, reference, additional, extra))

In [ ]:
from pyspark.sql import types as T

schema = T.StructType([
    T.StructField("Serial Number", T.StringType()),
    T.StructField("Part Number", T.StringType()),
    T.StructField("Part Name English", T.StringType()),
    T.StructField("Part Name Language 1", T.StringType()),
    T.StructField("Part Name Language 2", T.StringType()),
    T.StructField("Part Name Language 3", T.StringType()),
    T.StructField("Part Name Language 4", T.StringType()),
    T.StructField("Part Name Language 5", T.StringType()),
    T.StructField("Part Number in Other Format", T.StringType()),
    T.StructField("Reference Number", T.StringType()),
    T.StructField("Additional Information", T.StringType()),
    T.StructField("Extra Data", T.StringType())
])

df = spark.createDataFrame(final, schema)
df.show(truncate=False)

+-------------+---------------------------+-----------------------------------------------+----------------------------+-----------------------------------+-----------------------------+-----------------------------------------------------------------+-----------------------------------------------------------+---------------------------+----------------+----------------------+----------+
|Serial Number|Part Number                |Part Name English                              |Part Name Language 1        |Part Name Language 2               |Part Name Language 3         |Part Name Language 4                                             |Part Name Language 5                                       |Part Number in Other Format|Reference Number|Additional Information|Extra Data|
+-------------+---------------------------+-----------------------------------------------+----------------------------+-----------------------------------+-----------------------------+------------------------------

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T

# ---------------------------------------
# READ BINARY FILE
# ---------------------------------------
binary_df = spark.read.format("binaryFile").load("/content/input_file.txt")
raw_bytes = binary_df.select("content").first()[0]

# decode text
raw_text = raw_bytes.decode("latin1")

# normalize null-byte markers
raw_text = raw_text.replace("\\x00", "\x00")

# keep only ASCII + null bytes
import re
valid_text = re.findall(r'[\x00-\x7F]+', raw_text)[0]

# split fields
parts = [p for p in valid_text.split("\x00") if p.strip() != ""]

# ---------------------------------------
# DETECT NEW RECORD START
# ---------------------------------------

def is_new_record(idx):
    cur = parts[idx]

    # must be pure digits
    if not cur.isdigit():
        return False

    # serial numbers in your file are small (1–3 digits)
    if len(cur) > 4:
        return False

    return True


# ---------------------------------------
# BUILD RECORDS
# ---------------------------------------
records = []
current = []

i = 0
while i < len(parts):
    if is_new_record(i):
        if current:
            records.append(current)
        current = [parts[i]]
    else:
        current.append(parts[i])
    i += 1

if current:
    records.append(current)

# ---------------------------------------
# PARSE RECORDS → 12 COLUMNS
# ---------------------------------------
final = []

for r in records:
    # required minimal fields
    serial = r[0] if len(r) > 0 else None
    part_no = r[1] if len(r) > 1 else None
    english = r[2] if len(r) > 2 else None
    lang1 = r[3] if len(r) > 3 else None
    lang2 = r[4] if len(r) > 4 else None
    lang3 = r[5] if len(r) > 5 else None
    lang4 = r[6] if len(r) > 6 else None
    lang5 = r[7] if len(r) > 7 else None
    other_format = r[8] if len(r) > 8 else None
    reference = r[9] if len(r) > 9 else None
    additional = r[10] if len(r) > 10 else None

    # ---------------------------------------
    # Build EXTRA DATA (Option A)
    # repeating sets of 4 fields:
    # PN  | OtherFormat | Reference | Info
    # ---------------------------------------
    extra = None

    if len(r) > 11:
        tail = r[11:]
        grouped = []

        for i in range(0, len(tail), 4):
            block = tail[i:i+4]
            if len(block) == 4:
                grouped.append("_".join(block))

        extra = "_".join(grouped) if grouped else None

    final.append((
        serial, part_no, english, lang1, lang2, lang3, lang4, lang5,
        other_format, reference, additional, extra
    ))

# ---------------------------------------
# CREATE DATAFRAME
# ---------------------------------------
schema = T.StructType([
    T.StructField("Serial Number", T.StringType()),
    T.StructField("Part Number", T.StringType()),
    T.StructField("Part Name English", T.StringType()),
    T.StructField("Part Name Language 1", T.StringType()),
    T.StructField("Part Name Language 2", T.StringType()),
    T.StructField("Part Name Language 3", T.StringType()),
    T.StructField("Part Name Language 4", T.StringType()),
    T.StructField("Part Name Language 5", T.StringType()),
    T.StructField("Part Number in Other Format", T.StringType()),
    T.StructField("Reference Number", T.StringType()),
    T.StructField("Additional Information", T.StringType()),
    T.StructField("Extra Data", T.StringType())
])

df = spark.createDataFrame(final, schema)
df.show(truncate=False)


NameError: name 'spark' is not defined

In [ ]:
df.count()

57

In [ ]:
df.show(truncate=False)



+-------------+---------------------+---------------------------+-----------------------------------------------+------------------------------+-----------------------------------+---------------------------------------------------+-----------------------------------------------------------------+---------------------------+----------------+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Serial Number|Part Number          |Part Name English          |Part Name Language 1                           |Part Name Language 2          |Part Name Language 3               |Part Name Language 4                               |Part Name Language 5                                             |Part Number in Other Format|Reference Number|Additional Information     |Extra Data                                                               

In [ ]:
df = df.fillna("-")

df.show()

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+--------------------+
|Serial Number|         Part Number|   Part Name English|Part Name Language 1|Part Name Language 2|Part Name Language 3|Part Name Language 4|Part Name Language 5|Part Number in Other Format|Reference Number|Additional Information|          Extra Data|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+--------------------+
|            1|       X530108146000|INTERNAL RETAININ...|ANNEAU D\'ARRET P...|      INNENHALTERING|ANELLO DI SICUREZ...|ANILLO DE RETENCI...|PIER\xc5\x9aCIE\x...|           X530.108.146.000|        72311106|         DIN472 - 90X3|              

In [ ]:
# df.coalesce(1) \
#   .write \
#   .option("header", True) \
#   .option("encoding", "UTF-8") \
#   .mode("overwrite") \
#   .csv("/output_final.csv")

In [1]:
print("hello world here we are doing git hub trial")

hello world here we are doing git hub trial


In [ ]:
mf=spark.read.csv("/output.csv",header=True,inferSchema=True)
mf.show()

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+--------------------+
|Serial Number|         Part Number|   Part Name English|Part Name Language 1|Part Name Language 2|Part Name Language 3|Part Name Language 4|Part Name Language 5|Part Number in Other Format|Reference Number|Additional Information|          Extra Data|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+----------------+----------------------+--------------------+
|            1|       X530108146000|INTERNAL RETAININ...|ANNEAU D\'ARRET P...|      INNENHALTERING|ANELLO DI SICUREZ...|ANILLO DE RETENCI...|PIER\xc5\x9aCIE\x...|           X530.108.146.000|        72311106|         DIN472 - 90X3|              

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


# New Section